In [2]:
import csv
import pandas as pd
import random
import pyarrow.feather as feather
import regex as re
from cleantext import clean
import pyarrow.feather as feather
from multiprocessing import Pool
import gc
import nltk
import itertools
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import *
import time
import gc

In [3]:
sampled_data = pd.read_csv("data/sample.csv", on_bad_lines='skip')
#Loads the sample that is 10% of the full dataset

In [4]:
len(sampled_data)

1155872

In [5]:
biases = sampled_data[sampled_data['type'] == '” said Linton Roberts']
biases1 = biases.reset_index()
biases1.content[4]

' Cammell Laird."'

In [6]:
sputnik = biases1[biases1['content'] == 'It only takes a few moments to share an article, but the person on the other end who reads it might have his life changed forever.']
len(sputnik)

0

In [46]:
def removeLabels(df): 
    df.type = df.type.replace({'political': 'reliable', 'junksci': 'fake', 'bias' : 'fake', 'satire': 'fake', 'conspiracy': 'fake', 'rumor': 'fake', 'unreliable' : 'fake', 'clickbait': 'fake', 'hate': 'fake'})
    df = df[(df.type == 'reliable') |(df.type == 'fake')]
    df = df[df.type.notnull()]
    df = df[df.content.notnull()]
    df = df.drop_duplicates(subset = 'content', keep = 'last')
    df = df.reset_index()
    return df 

def labelperc(df): 
    labeldict = {}
    for i in df.type: 
        if i in labeldict: 
            labeldict[i] +=1
        else: 
            labeldict[i] = 1
    for i in labeldict: 
        labeldict[i] = labeldict[i]/len(df)*100
    return labeldict
cleanSample = removeLabels(sampled_data)
print(labelperc(cleanSample)) 
print(len(cleanSample))




{'reliable': 48.22415265633921, 'fake': 51.775847343660786}
635781


In [42]:
cleanSample.content[5]

'Market News | Thu Nov 3, 2016 | 7:25am EDT BRIEF-The Buckle qtrly sales fell 14.6 pct to $239.2 mln \nNov 3 Buckle Inc \n* Q3 revenue view $248.9 million -- Thomson Reuters I/B/E/S \n* The buckle, inc. Reports october 2016 net sales \n* Quarterly sales fell 14.6 percent to $239.2 million \n* October sales fell 15.1 percent to $69.1 million Source text for Eikon: Further company coverage: Next In Market News '

In [47]:
def checkForDuplicateArticles(df): 
    articledict = {}
    df_sample = df.sample(frac=0.1, random_state=1)
    for i in df_sample.content: 
        if str(i) in articledict: 
            articledict[str(i)] +=1
        else: 
            articledict[str(i)] = 1
    #at this point we have all different contents, and the amount of times we see them
    articledict = {key:val for key, val in articledict.items() if val >1}
    #here we delete the contents which we only see once

    contentsToRemove = list(articledict.keys())
    #get a list of the content, we will remove articles for containing
    
    
    return contentsToRemove
    
wordstoremove = checkForDuplicateArticles(cleanSample)
len(wordstoremove)




0